# django ORM cookbook part1

In [1]:
from django.contrib.auth import get_user_model
from django.db.models import Q
from events.models import *
from entities.models import *

## 1. ORM이 실행하는 실제 SQL query 확인

- django ORM이 실행하는 질의문 또는 작성한 코드에 대응하는 SQL query가 무엇인지 확인하고 싶을 때가 있다.
- SQL 질의문은 queryset.query의 str을 확인하면 된다.

In [2]:
queryset = Event.objects.all()
SQL_query = str(queryset.query)
print(SQL_query)

SELECT "events_event"."id", "events_event"."epic_id", "events_event"."details", "events_event"."years_ago" FROM "events_event"


In [3]:
queryset = Event.objects.filter(years_ago__gt=5)
SQL_query = str(queryset.query)
print(SQL_query)

SELECT "events_event"."id", "events_event"."epic_id", "events_event"."details", "events_event"."years_ago" FROM "events_event" WHERE "events_event"."years_ago" > 5


## 2. OR연산 이용하기

- 여러 조건 중 하나만 만족해도 되는 data를 들고와야 하는 경우가 있다.
- SQL query로는 WHERE와 OR을 함께 이용하거나 WHERE와 UNION을 이용하여 구할 수 있다.

In [4]:
User = get_user_model()

queryset = User.objects.filter(
    first_name__startswith='R'
) | User.objects.filter(
    last_name__startswith='D'
)

SQL_query = str(queryset.query)
print(SQL_query)

SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE '\' OR "auth_user"."last_name" LIKE D% ESCAPE '\')


- Q객체를 이용하는 방법
    - 이전의 ORM과 동일한 query를 가진다.

In [5]:
queryset = User.objects.filter(
    Q(first_name__startswith='R') |
    Q(last_name__startswith='D')
)

SQL_query = str(queryset.query)
print(SQL_query)

SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE '\' OR "auth_user"."last_name" LIKE D% ESCAPE '\')


- Union을 이용하는 방법
    - 데이터셋이 큰 경우 OR연산보다 Union을 이용하는 경우가 더 빠르다

In [6]:
queryset_1 = User.objects.filter(first_name__startswith='R')
queryset_2 = User.objects.filter(last_name__startswith='D')

queryset = queryset_1.union(queryset_2)

SQL_query = str(queryset.query)
print(SQL_query)

SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE "auth_user"."first_name" LIKE R% ESCAPE '\' UNION SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE "auth_user"."last_name" LIKE D% ESCAPE '\'


3. AND연산 이용하기
- 여러 조건을 모두 만족하는 데이터를 가져와야 할 때 AND 연산을 이용한다.
- SQL query로는 WHERE과 AND를 함께 사용하는 경우이다.

In [7]:
queryset = User.objects.filter(
    first_name__startswith='R',
    last_name__startswith='D'
)

SQL_query = str(queryset.query)
print(SQL_query)

SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE '\' AND "auth_user"."last_name" LIKE D% ESCAPE '\')


In [8]:
queryset = User.objects.filter(
    Q(first_name__startswith='R') &
    Q(last_name__startswith='D')
)

SQL_query = str(queryset.query)
print(SQL_query)

SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE '\' AND "auth_user"."last_name" LIKE D% ESCAPE '\')


4. NOT 연산으로 조건 부정하기
- 특정 조건을 만족하지 않는 데이터를 가져와야 할 때 NOT을 이용한다.
- `exclude`나 `~Q()`를 이용한다.
- 역방향 참조모델에 대한 NOT 연산은 django ORM상 sub query가 발생하므로 유의해서 사용하자.(https://www.youtube.com/watch?v=EZgLfDrUlrk)

In [9]:
queryset = User.objects.exclude(id__lt=5)

SQL_query = str(queryset.query)
print(SQL_query)

SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE NOT ("auth_user"."id" < 5)


In [10]:
queryset = User.objects.filter(
    ~Q(id__lt=5)
)

SQL_query = str(queryset.query)
print(SQL_query)

SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE NOT ("auth_user"."id" < 5)


**참고**
https://stackoverflow.com/questions/15361972/why-is-union-faster-than-an-or-statement
https://www.youtube.com/watch?v=EZgLfDrUlrk
https://django-orm-cookbook-ko.readthedocs.io/en/latest/notequal_query.html